Introduction:
===========
**Coronavirus Disease 2019 (COVID-19)** was first identified in December 2019 in Wuhan, the capital of China's Hubei province and has since then spread to be identified as a Global Pandemic.

What is COVID-19:
-----------------
COVID-19 is a novel coronavirus which expresses itself in humans as a respiratory illness.

How does it spread:
----------------------------
According to the World Health Organization as of 15/04/2020 "This disease can spread from person to person through small droplets from the nose or mouth which are spread when a person with COVID-19 coughs or exhales. These droplets land on objects and surfaces around the person. Other people then catch COVID-19 by touching these objects or surfaces, then touching their eyes, nose or mouth. People can also catch COVID-19 if they breathe in droplets from a person with COVID-19 who coughs out or exhales droplets."

The question that is on everyones mind is **"How could we have prevented the spread?"**
We hope to allow the research to answer this.

What can we do to stop it:
--------
*For the purpose of this challenge we will be focusing on Nonpharmaceutical Interventions in preventing the spread of COVID-19.*

As per the CDC: **Nonpharmaceutical Interventions(NPIs)** are actions, apart from getting vaccinated and taking medicine, that people and communities can take to help slow the spread of illnesses like pandemic influenza (flu).

NPI's are typically broken up into 3 categories listed with examples:

    1. Personal: 
        -Staying home when you are sick
        -Covering coughs and sneezes with a tissue
        -Washing hands with soap and water or using hand sanitizer when soap and water is not available
    2. Community:
        -Social Distancing
        -Closures (child care centers, schools, places of worship, sporting events, etc)
    3. Environmental:
        -Routine surface cleaning 

The goal of our submission is to analyze the research done on the implementation, efficacy and scaling of NPI's, isolate these papers from other COVID-19 research articles and derive meaningful insights to help us mitigate the existing situaiton and prepare our systems for any future pandemic which may come our way.   



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import json

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 100)
pd.set_option('display.max_colwidth', -1)

C:\Users\tnd005\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [3]:
# data csv from Kaggle notebook instead of pickling
df = pd.read_csv('data.csv')

In [4]:
len(df)

1301

In [5]:
df.head()

Unnamed: 0                                  paper_id  \
0  0           b891efc6e1419713b05ff7d89b26d260478c28df   
1  1           7a5d731cd59597da7bcef171af936c0af4d7528c   
2  2           854545208a6bc7cafa8744604899e32237ff2047   
3  3           e8d81153777e9f92cbc2bc9ea03ed1f0d0b78543   
4  4           a14b5655cb13ed64cb8cff7c806a7b58c858b8b7   

                                                                                                                                                                                           title  \
0  6756    tuberculosis prevention in healthcare workers in china  years after the severe acute respiratory syndrome pandemic\r\nName: title, dtype: object                                        
1  epidemic and case investigations surveillance operation for the 141st confirmed case of middle east respiratory syndrome coronavirus in response to the patient's prior travel to jeju island   
2  highly pathogenic avian influenza a virus h5n1 non-structural protein 1 is associated with apoptotic activation of the intrinsic mitochondrial pathway                                          
3  group b betacoronavirus in rhinolophid bats, japan                                                                                                                                              
4  feasibility of controlling covid-19 outbreaks by isolation of cases and contacts                                                                                                                

                                                                                                                                                                                                                                                                                                              author_list  \
0  []                                                                                                                                                                                                                                                                                                                       
1  ['Open Access', 'Jong-Myon Bae']                                                                                                                                                                                                                                                                                         
2  ['Qian Bian', 'Jing Lu', 'L I Zhang', 'Ying Chi', 'Yan Li', 'Hongxiong Guo']                                                                                                                                                                                                                                             
3  ['Jin Suzuki', 'Ryota Sato', 'Tomoya Kobayashi', 'Toshiki Aoi', 'Ryô Harasawa']                                                                                                                                                                                                                                          
4  ['J Hellewell', 'S Abbott', 'Jarvis Phd', 'T W Russell', 'J D Munday', 'A J Kucharski', 'W J Edmunds', 'S Funk', 'Rosalind M Eggo', 'Joel Hellewell', 'Sam Abbott', 'Amy Gimma', 'Nikos I Bosse', 'Christopher I Jarvis', 'Timothy W Russell', 'James D Munday', 'Adam J Kucharski', 'John Edmunds', 'Sebastian Funk']   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [6]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [7]:
class Preprocesser:
    def __init__(self, df):
        self.keywords = ['incident command system',
                         'emergency operations',
                         'joint information center',
                         'social distancing',
                         'childcare closers',
                         'travel advisory',
                         'travel warning',
                         'isolation',
                         'quarantine',
                         'mass gathering cancellations',
                         'school closures',
                         'facility closures',
                         'evacuation',
                         'relocation',
                         'restricting travel',
                         'travel ban',
                         'patient cohort',
                         'npi']
        self.occurances_minimum = 2
        self.df_full = df
        print(self.df_full.shape)
        self.key_slice()
        print(self.df_full.shape)
        self.npi_slice()
        print(self.df_full.shape)
    
    def key_slice(self):
        self.df_full = self.df_full[self.df_full['abstract'].str.contains('|'.join(self.keywords), na=False, regex=True)].reset_index(drop=True)
        
    def npi_slice(self):
        def get_count(row):
            return sum([row['abstract'].count(keyword) for keyword in self.keywords])
        self.df_full = self.df_full[self.df_full.apply(get_count, axis=1) >= self.occurances_minimum]
        
    def remove_stopwords(self,columns):
        stop = stopwords.words('english')
        for col in columns:
            self.df_full[col] = self.df_full[col].astype(str).apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

    def to_tfidf(self,columns):
        for col in columns:
            tfidfv = TfidfVectorizer()
            self.df_full[col + '_tfidf'] = list(tfidfv.fit_transform(self.df_full[col]).toarray())
            
    def remove_punc(self, columns):
        for col in columns:
            self.df_full[col] = self.df_full[col].str.replace('[^a-zA-Z\s]+','')
        
def display_wordcloud(text):
    wordcloud = WordCloud(max_font_size=50, max_words=100, background_color='white').generate(text)
    plt.figure()
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

In [8]:
def pca_apply(df, columns, n_comp):
    new_df = df.copy()
    for col in columns:
        pca = PCA(n_components=n_comp, random_state=1)
        new_df[col+'_pca'] = list(pca.fit_transform(np.stack(df[col].to_numpy())))
    return new_df.reset_index(drop=True)

def apply_scaler(df, columns):
    new_df = df.copy()
    for col in columns:
        scaler = StandardScaler()
        new_df[col + '_scaled'] = list(scaler.fit_transform(np.stack(df[col].to_numpy())))
    return new_df.reset_index(drop=True)

In [9]:
prepr = Preprocesser(df)

(1301, 7)
(1301, 7)
(428, 7)


In [10]:
prepr.remove_punc(['body_text','abstract'])
prepr.remove_stopwords(['body_text', 'abstract'])
prepr.to_tfidf(['body_text', 'abstract'])
pca_df = pca_apply(prepr.df_full, ['abstract_tfidf','body_text_tfidf'], 3)
scaled_df = apply_scaler(pca_df,['abstract_tfidf_pca','body_text_tfidf_pca'])
# clustered_df = cluster(scaled_df, ['abstract_tfidf_pca_scaled', 'body_text_tfidf_pca_scaled'], 10)

In [ ]:
# from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output

def get_breaks(row, col, word_limit=45, break_char='<br>', colon=True):
    if row[col] == row[col]:
        words = row[col].split(' ')
        if colon:
            data = f'{col}: '
        else:
            data.replace('<br>', '')
        total_chars = 0

        # add break every length characters
        for i in range(len(words)):
            total_chars += len(words[i])
            if total_chars > word_limit:
                data += f'{break_char}{words[i]}'
                total_chars = 0
            else:
                data += f' {words[i]}'
        return data
    return row[col]

class Cluster_Plot:
    def __init__(self, df, text_type, clust_nums):
        self.styles = {
            'pre': {
                'border': 'thin lightgrey solid',
                'overflowX': 'scroll'
            }
        }
        self.set_app()
        self.df = df
        self.set_text(text_type)
        self.clust_nums = clust_nums
        self.create_cluster_df()
        
        if self.app is not None and hasattr(self, 'callbacks'):
            self.callbacks(self.app)

    def run_process(self):
        self.set_app_layout()
        self.app.run_server()
        
    def set_app(self):
        self.app = dash.Dash(__name__,
                        external_stylesheets=["https://codepen.io/chriddyp/pen/bWLwgP.css", "../localstyles.css"])
        
    def set_app_layout(self):
        self.app.layout = html.Div(children=[
            html.Div(className='row', style={'background-color': '#142a57'}, children=[
                html.Div(className='container', style={'max-width': 'unset'}, children=[
                    html.Div(className='row', children=[
                        html.Div(className='nine columns', children=[
                            html.H1('NPI Cluster Analysis', style={ 'color': 'white', 'padding-top': '1%'})
                        ]),
                        html.Div(className='three columns', children=[
                            html.Img(
                                src='https://storage.googleapis.com/lab_assets/Jefferson_CorporateEnterpriseLightBlue-01.png',
                                style={ 'width': '60%', 'padding-top': '5%'}
                            )
                        ])
                    ])
                ])
            ]),
            html.Div(className='container', style={'max-width': 'unset'}, children=[
                html.Div(className='row', children=[
                    html.Div(className='three columns', children=[
                        html.H2('Cluster Control Panel'),
                        html.Label('Select to cluster on the article abstract or body'),
                        dcc.RadioItems(
                            id='abstract_or_body',
                            options=[{'label': 'abstract', 'value': 'abstract'},
                                     {'label': 'body', 'value': 'body_text'}],
                            value=self.text_type
                        ),
                        html.Label('Select number of Clusters'),
                        dcc.Dropdown(
                            id='cluster_num',
                            options=[{'label': i+1, 'value': i+1} for i in list(range(20))],
                            value=self.clust_nums
                        ),
                        html.Label('Select cluster to view'),
                        dcc.Dropdown(
                            id='cluster_id',
                            options=self.cluster_id_list,
                            value='all'
                        ),
                    ]),
                    html.Div(className='six columns', children=[
                        dcc.Graph(id="graph", style={"width": "90%", "display": "inline-block"})
                    ]),
                    html.Div(className='three columns', children=[
                        html.H2('Filter Articles'),
                        html.Label('Search keywords'),
                        dcc.Input(id='search',
                            value='',
                            type='text'
                        )
                    ]),
                ]),
                html.Div(className='row', children=[
                    html.Div([
                        dcc.Markdown("""
                            **Selected Article**

                            Click on values in the plot to select article.
                        """),
                        html.Pre(id='hover-data', style=self.styles['pre'])
                    ])
                ])
            ]),
            html.Div(className='row', style={'background-color': '#142a57', 'position': 'fixed', 'bottom': '0'}, children=[
                html.Div(style={'max-width': 'unset'}, children=[
                    html.Img(
                                src='https://storage.googleapis.com/lab_assets/DICE_PoweredByEnhancedWhite.png',
                                style={ 'width': '10%', 'padding-top': '1%', 'margin-left': '45%', 'margin-right': '45%'}
                            )
                ])
            ])
        ])
        
    def create_cluster_df(self):
        self.cluster_id_list = [{'label': i, 'value': i} for i in list(range(self.clust_nums))]
        self.cluster_id_list.append({'label': 'all', 'value': 'all'})
        new_df = self.df.copy()
        kmeans = KMeans(n_clusters = self.clust_nums, random_state=1)
        new_df[self.col_cluster_id] = list(kmeans.fit_predict(np.stack(new_df[self.col].to_numpy())))
        self.cluster_df = new_df.reset_index(drop=True)
        self.cluster_df['title'] = self.cluster_df.apply(get_breaks, args=('title',), axis=1)
        self.cluster_df[['x', 'y', 'z']] = pd.DataFrame(self.cluster_df[self.col].values.tolist(),
                                                        index = self.cluster_df.index)
    
    def set_text(self, text_type):
        self.text_type = text_type
        self.col = f'{self.text_type}_tfidf_pca_scaled'
        self.col_cluster_id = f'{self.text_type}_tfidf_pca_scaled_clusterID'
        
    def callbacks(self, app):
        @app.callback([Output('cluster_id', 'options'),
                       Output('graph', 'figure')],
                      [Input('abstract_or_body', 'value'),
                       Input('cluster_num', 'value'),
                       Input('cluster_id', 'value'),
                       Input('search', 'value')])
        def update_clusters(abstract_or_body, cluster_num, cluster_id, search_string):
            self.set_text(abstract_or_body)
            self.clust_nums = cluster_num
            self.create_cluster_df()
            options = self.cluster_id_list
            df = self.cluster_df.copy()
            show_scale = True
            if cluster_id != 'all':
                show_scale = False
                df = df[df[self.col_cluster_id] == cluster_id]
            df = df[df[self.text_type].str.contains(search_string, na=False, regex=True)].reset_index(drop=True)
            fig = px.scatter_3d(df, x='x', y='y', z='z',
                                 color=self.col_cluster_id,
                                 hover_name='title',
                                 hover_data=['paper_id', 'doi'])
            fig.update_layout(scene = dict(
                                xaxis = dict(nticks=4, range=[-5,5],),
                                yaxis = dict(nticks=4, range=[-5,5],),
                                zaxis = dict(nticks=4, range=[-5,5],),),
                              hoverlabel=dict(
                                bgcolor='white', 
                                font_size=8, 
                                font_family='Rockwell'
                              ),
                              coloraxis=dict(
                                colorbar=dict(title='Cluster ID'),
                                showscale=show_scale
                              ))
            return options, fig
        
        @app.callback(Output("hover-data", "children"), [Input("graph", "clickData")])
        def display_click_data(clickData):
            string = None
            if clickData:
                print(clickData['points'][0])
                click_index = clickData['points'][0]['pointNumber']
                string = f'{self.cluster_df.iloc[click_index]["title"]}'
                item_list = ['abstract', 'body_text', 'author_list', 'paper_id', 'doi']
                for i in item_list:
                    formatted_data = get_breaks(self.cluster_df.iloc[click_index],
                                               i,
                                               word_limit=100,
                                               break_char='\n')
                    string += f'\n\n{formatted_data}'
            return string

c = Cluster_Plot(scaled_df, 'abstract', 10)
c.run_process()

 * Tip: There are .env or .flaskenv files present. Do "pip install python-dotenv" to use them.


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
